# Сравнение поведения групп пользователей музыкального стримингового сервиса

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

Согласно заданию статистическая проверка гипотез не требовалась.

Данные о поведении пользователей содержат следующую информацию:
* userID — идентификатор пользователя;
* Track — название трека;  
* artist — имя исполнителя;
* genre — название жанра;
* City — город пользователя;
* time — время начала прослушивания;
* Day — день недели.

План исследования
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных



In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('f:/Yandex-Practicum/datasets/yandex_music_project.csv')

In [3]:
# получение первых 10 строк таблицы df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


**Выводы**

В таблице 65079 записей с семью столбцами. Тип данных во всех столбцах — object. Каждая запись — данные о прослушанном треке. Часть колонок описывает прослушанный трек: название, исполнителя и жанр, остальные данные рассказывают о пользователе: его город и дата прослушивания трека. 


Данных достаточно для проверки гипотез, но в данных встречаются пропуски, а названия столбцов содержат лишние пробелы.

## Предобработка данных
Исправление стиля заголовков столбцов, исключение пропусков, проверка данных на дубликаты.

### Стиль заголовков

In [5]:
# перечень названий столбцов таблицы df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Названия столбцов не в змеинов регистре, в названиях содержаться лишние пробелы

In [6]:
# переименование столбцов в змеиный регистр
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city','Day': 'day'})

In [7]:
# проверка переименования столбцов
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений

In [8]:
# подсчёт пропусков
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

В столбцах track, artist и genre присутствуют пропуски. Первые два столбца не используются для сравнения групп пользователей. Все пропуски заполняются явными обозначениями - 'unknown' для сравнения доли пропусков в городах.

In [9]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
df[columns_to_replace] = df[columns_to_replace].fillna('unknown')

In [10]:
# проверка заполнения, подсчёт пропусков
df.isna().sum().max()

0

In [11]:
# Часть пропусков можно попробовать восстановить по названию трека или исполнителю, строим распределение всех и "восстанавливаемых" пропусков.
#Распределение всех пропусков жанра по городам и дням недели
df_without_genre=df[df['genre'] == 'unknown']
print('Распределение пропусков жанра по городам и дням недели (всего', df_without_genre['user_id'].count(), ')\n', 
      df_without_genre.groupby('city')['city'].count(), '\n', 
      df_without_genre.groupby('day')['day'].count()
     )
# Распределение по городам и дням недели для "восстанавливаемых" пропусков жанра 
print('\nРаспределение по городам и дням недели для "восстанавливаемых" пропусков (всего ', 
      df_without_genre[(df_without_genre['track'] != 'unknown') 
                       | (df_without_genre['artist'] != 'unknown')]['user_id'].count(),  ')\n',
      df_without_genre[(df_without_genre['track'] != 'unknown')
                       | (df_without_genre['artist'] != 'unknown')].groupby('city')['city'].count(), '\n',
      df_without_genre[(df_without_genre['track'] != 'unknown') 
                       | (df_without_genre['artist'] != 'unknown')].groupby('day')['day'].count()
     )

Распределение пропусков жанра по городам и дням недели (всего 1198 )
 city
Moscow              904
Saint-Petersburg    294
Name: city, dtype: int64 
 day
Friday       375
Monday       516
Wednesday    307
Name: day, dtype: int64

Распределение по городам и дням недели для "восстанавливаемых" пропусков (всего  537 )
 city
Moscow              395
Saint-Petersburg    142
Name: city, dtype: int64 
 day
Friday       205
Monday       179
Wednesday    153
Name: day, dtype: int64


### Дубликаты

In [12]:
# подсчёт явных дубликатов
df.duplicated().sum()

3826

In [13]:
# удаление явных дубликатов с удалением старых индексов и формированием новых
df = df.drop_duplicates().reset_index(drop=True)

In [14]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

В столбце жанр могут присутствовать неявные дубликаты, если название одного и того же жанра записано по-разному.

In [15]:
# Просмотр уникальных названий жанров
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

In [16]:
# Функция для замены неявных дубликатов
def replace_wrong_genres(wrong_genres,correct_genre):
    for i in wrong_genres:
        df['genre'] = df['genre'].replace(i, correct_genre)

In [17]:
# Устранение неявных дубликатов
replace_wrong_genres(['hip', 'hop', 'hip-hop'], 'hiphop')
# Дополнительно устраняется дублирование
replace_wrong_genres(['электроника'], 'electronic')


**Нужно дополнительно уточнить у заказчика о дублировании следующих жанров**
* (['poprussian'], 'ruspop')
* (['türk'], 'türkçe')
* ['latino'], 'latin')
* (['frankreich', 'französisch'], 'french')
* ['neue'], 'new')

In [18]:
# Проверка встречаемости дублирующихся жанров
duplicated_genre_name = ['poprussian', 'ruspop', 
                          'türk', 'türkçe',
                         'latino', 'latin',
                          'frankreich', 'französisch', 'french',
                          'neue', 'new'
                         ]
print('Количество использований дублирующихся жанров:')
for i in duplicated_genre_name:
    print(i, '-', df[df['genre'] == i]['genre'].count())

Количество использований дублирующихся жанров:
poprussian - 1
ruspop - 1910
türk - 4
türkçe - 25
latino - 2
latin - 921
frankreich - 1
französisch - 1
french - 83
neue - 1
new - 568


In [19]:
# Переименование редко встречающихся дублирующихся жанров на часто встречающиеся
replace_wrong_genres(['poprussian'], 'ruspop')
replace_wrong_genres(['türk'], 'türkçe')
replace_wrong_genres(['latino'], 'latin')
replace_wrong_genres(['frankreich', 'französisch'], 'french')
replace_wrong_genres(['neue'], 'new')

In [20]:
# Проверка жанров после замены их неявных дубликатов
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Заголовки переведены в змеиный регистр.

Пропуски заполнены значениями 'unknown', определено количество пропусков для разных городов. Примерно половина записей с отсутствующим жанром содержит название и/или исполнителя, по которым можно попробовать восстановить жанр.

Дубликаты явные удалены, а также обработаны неявные дубликаты с разным написанием одного жанра.

In [21]:
#анализ пропусков после удаления явных дубликатов
#Распределение пропусков жанра по городам и дням недели
df_without_genre=df[df['genre'] == 'unknown']
print('Распределение пропусков жанра по городам и дням недели (всего', df_without_genre['user_id'].count(), ')\n', 
df_without_genre.groupby('city')['city'].count(), '\n', 
df_without_genre.groupby('day')['day'].count())
# Распределение по городам и дням недели для "восстанавливаемых" пропусков жанра 
print('\nРаспределение по городам и дням недели для "восстанавливаемых" пропусков (всего ', 
      df_without_genre[(df_without_genre['track'] != 'unknown') 
                       | (df_without_genre['artist'] != 'unknown')]['user_id'].count(),  ')\n',
      df_without_genre[(df_without_genre['track'] != 'unknown')
                       | (df_without_genre['artist'] != 'unknown')].groupby('city')['city'].count(), '\n',
      df_without_genre[(df_without_genre['track'] != 'unknown') 
                       | (df_without_genre['artist'] != 'unknown')].groupby('day')['day'].count())


Распределение пропусков жанра по городам и дням недели (всего 1127 )
 city
Moscow              849
Saint-Petersburg    278
Name: city, dtype: int64 
 day
Friday       358
Monday       488
Wednesday    281
Name: day, dtype: int64

Распределение по городам и дням недели для "восстанавливаемых" пропусков (всего  508 )
 city
Moscow              377
Saint-Petersburg    131
Name: city, dtype: int64 
 day
Friday       193
Monday       172
Wednesday    143
Name: day, dtype: int64


In [22]:
# поиск треков с неизвестным жанром, у исполнителей которых есть треки с известным жанром
res = pd.DataFrame()
genre_dictionary = {} # словарь для восстановления жанра по автору
for i in df_without_genre['artist'].unique():
    if ((i != 'unknown') and (len(df[(df['artist'] == i) & (df['genre'] != 'unknown')]) != 0)):
        res = pd.concat([res, df.query('@i==artist')])

res.head(10)#.groupby('genre')['genre'].count().sort_values()

,user_id,track,artist,genre,city,time,day
613,647B7722,Thing Called Love,Wolfram,unknown,Moscow,20:23:26,Monday
17097,E166EE5B,A Thing Called Love,Wolfram,indie,Saint-Petersburg,21:43:28,Friday
1378,C1C70D5C,Only for Tonight,Late Night Alumni,unknown,Moscow,20:15:09,Friday
42762,49602641,Lament,Late Night Alumni,electronic,Saint-Petersburg,09:55:56,Monday
47793,58909942,Empty Streets,Late Night Alumni,dance,Saint-Petersburg,13:17:19,Wednesday
2689,981145D0,Ashes,Grace Carter,unknown,Saint-Petersburg,21:47:47,Monday
5323,EE3159FD,Why Her Not Me,Grace Carter,pop,Saint-Petersburg,08:07:36,Wednesday
644,29E6B1CF,Among Them,Dreamy,electronic,Saint-Petersburg,14:25:07,Monday
5514,D8AEED92,If I Knew,Dreamy,unknown,Moscow,20:49:57,Friday
10803,6C9E112E,A Man For All Seasons,Al Stewart,unknown,Saint-Petersburg,08:01:36,Monday


Проверено предположение о возможности восстановления жанра трека по исполнителю и названию. Получена таблица треков с неизвестным жанром, у исполнителей которых есть треки с известным жанром. В результате анализа выявлены исполнители, треки которых имеют разные жанры, например, у "Late Night Alumni" есть треки жанров electronic и dance. <b> Это значит, что восстановление жанра по автору не может быть выполнено однозначно и достоверно. </b> Кроме того, кол-во таких треков мало, и в лучшем случае можно было бы восстановить около 5 % треков от общего числа треков с неизвестным жанром.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая проверяемая гипотеза: пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Необходимо проверить это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. 

План:

* Разделить пользователей Москвы и Санкт-Петербурга
* Оценить активность пользователей в каждом городе, сгруппировав данные по городу и рассчитав прослушивания в каждой группе
* Сравнить, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

In [23]:
# Подсчёт прослушиваний в каждом городе
df.groupby('city')['time'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: time, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку, так как самих пользователей в Москве больше.

In [24]:
# Подсчёт прослушиваний в каждый из трёх дней
df.groupby('day')['time'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: time, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но необходимо рассмотреть города по отдельности.

In [25]:
# создание функции number_tracks
def number_tracks(day,city):
    """    Функция возвращает число прослушиваний для конкретного города и дня.
    
    С помощью последовательной фильтрации с логической индексацией 
    получает из исходной таблицы строки с нужным днём,
    затем из результата фильтрует строки с нужным городом,
    методом count() расчитывает количество значений в колонке user_id. 
    Это количество функция возвращает в качестве результата

    """    
    track_list = df[(df['day'] == day) & (df['city'] == city)]
    track_list_count = track_list['user_id'].count()  
    return track_list_count

In [26]:
# количество прослушиваний по городам и дням недели

# Англо-русский словарь для печати
dictionary = { 
    'Moscow': 'Москва', 
    'Saint-Petersburg': 'Санкт-Петербург', 
    'Friday': 'пятницу', 
    'Monday': 'понедельник', 
    'Wednesday': 'среду'}
for j in ['Monday', 'Wednesday', 'Friday']:
    for i in ['Moscow', 'Saint-Petersburg']:
        
        print(f'Количество прослушиваний в городе {dictionary[i]} в {dictionary[j]} - {number_tracks(j,i)}')

Количество прослушиваний в городе Москва в понедельник - 15740
Количество прослушиваний в городе Санкт-Петербург в понедельник - 5614
Количество прослушиваний в городе Москва в среду - 11056
Количество прослушиваний в городе Санкт-Петербург в среду - 7003
Количество прослушиваний в городе Москва в пятницу - 15945
Количество прослушиваний в городе Санкт-Петербург в пятницу - 5895


In [27]:
# Таблица с результатами
tracks_in_city_on_day = [['Moscow'], ['Saint-Petersburg']]
for i in range(2):
    for j in ['Monday', 'Wednesday', 'Friday']:
        tracks_in_city_on_day[i].append(number_tracks(j,tracks_in_city_on_day[i][0]))

pd.DataFrame(data=tracks_in_city_on_day,columns=['city', 'monday', 'wednesday', 'friday'])

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают различное поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Вторая проверяемая гипотеза: утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 

In [28]:
# получение таблицы пользователей из Москвы
moscow_general = df[df['city'] == 'Moscow']

In [29]:
# получение таблицы пользователей из СПБ
spb_general = df[df['city'] == 'Saint-Petersburg']

In [30]:
def genre_weekday(table, day, time1, time2):
    """
    возвращает информацию о 10 самых популярных жанрах в указанный день в заданное промежуток времени
    параметры
    * таблица (датафрейм) с данными,
    * день недели,
    * начальная временная метка в формате 'hh:mm', 
    * последняя временная метка в формате 'hh:mm'.
    """
    genre_df = table.query('@day==day and @time1 < time < @time2')
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_count = genre_df_count.sort_values(ascending=False)
    return genre_df_count.head(10)

# вывод процента от общего числа
def genre_weekday_percents(table, day, time1, time2):
    genre_df = table.query('@day==day and @time1 < time < @time2')
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_count = genre_df_count.sort_values(ascending=False)
    return (genre_df_count.head(15)/genre_df_count.sum()*100).round(decimals=1) #отбросить дробную часть

In [31]:
# Просмотр распределения количества записей в течение суток
for i in range(24):
    if i <= 8:
        ii = f'0{i}'
        jj = f'0{i+1}'
    elif i == 9:
        ii = f'0{i}'
        jj = f'{i+1}'
    else:
        ii = f'{i}'
        jj = f'{i+1}'
    print(f'кол-во треков c {ii} до {jj} ч.', 
          df[(df['time'] >= f'{ii}:00') & (df['time'] < f'{jj}:00')]['time'].count()
         )

кол-во треков c 00 до 01 ч. 0
кол-во треков c 01 до 02 ч. 0
кол-во треков c 02 до 03 ч. 0
кол-во треков c 03 до 04 ч. 0
кол-во треков c 04 до 05 ч. 0
кол-во треков c 05 до 06 ч. 0
кол-во треков c 06 до 07 ч. 0
кол-во треков c 07 до 08 ч. 0
кол-во треков c 08 до 09 ч. 9512
кол-во треков c 09 до 10 ч. 9511
кол-во треков c 10 до 11 ч. 169
кол-во треков c 11 до 12 ч. 0
кол-во треков c 12 до 13 ч. 0
кол-во треков c 13 до 14 ч. 10304
кол-во треков c 14 до 15 ч. 10722
кол-во треков c 15 до 16 ч. 176
кол-во треков c 16 до 17 ч. 0
кол-во треков c 17 до 18 ч. 0
кол-во треков c 18 до 19 ч. 0
кол-во треков c 19 до 20 ч. 0
кол-во треков c 20 до 21 ч. 10829
кол-во треков c 21 до 22 ч. 9858
кол-во треков c 22 до 23 ч. 172
кол-во треков c 23 до 24 ч. 0


**В датасете данные только в диапазонах с 08:00 до 10:01, с 13:00 до 15:01, с 20:00 до 22:01.**

Сравнение результатов функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [32]:
# вызов функции для утра понедельника в Москве 
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [33]:
# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [34]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [35]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

In [36]:
print('\nРаспределение жанров в городах по утрам понедельника, %')
pd.concat([genre_weekday_percents(moscow_general, 'Monday', '07:00', '11:00'),
           genre_weekday_percents(spb_general, 'Monday', '07:00', '11:00')], 
          axis=1, sort=False, keys=['Moscow', 'Saint-Petersburg'])


Распределение жанров в городах по утрам понедельника, %


,Moscow,Saint-Petersburg
genre,,
pop,14.9,13.6
dance,10.4,11.3
electronic,9.1,9.2
rock,9.0,10.1
hiphop,5.4,5.0
ruspop,3.5,4.0
world,3.4,2.2
rusrap,3.3,3.4
alternative,3.1,3.6


In [37]:
print('\nРаспределение жанров в городах в среду в обед, %')
pd.concat([genre_weekday_percents(moscow_general, 'Wednesday', '12:00', '16:00'),
           genre_weekday_percents(spb_general, 'Wednesday', '12:00', '16:00')], 
          axis=1, sort=False, keys=['Moscow', 'Saint-Petersburg'])


Распределение жанров в городах в среду в обед, %


,Moscow,Saint-Petersburg
genre,,
pop,13.8,14.3
dance,10.7,9.8
rock,9.4,10.5
electronic,8.3,8.4
classical,4.5,3.6
hiphop,4.2,4.9
world,3.4,2.9
alternative,3.0,3.1
ruspop,3.0,2.9


In [38]:
print('\nРаспределение жанров в городах по вечерам пятницы, %')
pd.concat([genre_weekday_percents(moscow_general, 'Friday', '17:00', '23:00'),
           genre_weekday_percents(spb_general, 'Friday', '17:00', '23:00')], 
          axis=1, sort=False, keys=['Moscow', 'Saint-Petersburg'])


Распределение жанров в городах по вечерам пятницы, %


,Moscow,Saint-Petersburg
genre,,
pop,13.5,12.4
rock,9.8,10.5
dance,9.4,10.2
electronic,9.1,10.5
hiphop,5.2,4.7
world,3.9,2.6
ruspop,3.2,2.3
alternative,3.1,3.0
classical,3.1,2.9


**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение 'unknown' заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных, что снижает достоверность исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные данные о жанрах.


Дополнительно рассмотрены жанры в среду в обед. В Москве популярность классической музыкы и джаза заметно растет (примерно в 1,5 раза), в Питере рост не наблюдается.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [39]:
# группировка таблицы moscow_general по столбцу 'genre', подсчёт числа значений 'genre' в этой группировке, сортировка в порядке убывания
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [40]:
# просмотр первых 10 строк moscow_genres
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Теперь повторите то же и для Петербурга.

Сгруппируйте таблицу `spb_general` по жанру. Посчитайте прослушивания треков каждого жанра. Результат отсортируйте в порядке убывания и сохраните в таблице `spb_genres`:


In [41]:
# группировка таблицы spb_general по столбцу 'genre', подсчёт числа значений 'genre' в этой группировке, сортировка в порядке убывания
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [42]:
# просмотр первых 10 строк spb_genres
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          539
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки гипотезе, рэп одинаково популярен в Москве и Петербурге (5 место). 


## Итоги исследования

Проверены три гипотезы и установлено:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Но по вечерам пятницы в Петербурге русский поп вылетает из топ 10, а в Москве джаз не входит в топ 10.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.